# Setup

In [ ]:
!pip install tqdm
!pip install littleballoffur

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 12.5 MB/s eta 0:00:00
  Created wheel for littleballoffur: filename=littleballoffur-2.3.1-py3-none-any.whl size=40737 sha256=8303c02ee85f48949a6d848cd886c2a5cf3ed2abea394eed4242ebd55b85da6b
  Stored in directory: /root/.cache/pip/wheels/f8/60/ca/ca2f94552c4789b79e4771b317251d42a63d36135923f9e791
Successfully built littleballoffur
  Attempting uninstall: cmake
    Found existing installation: cmake 3.25.2
    Uninstalling cmake-3.25.2:
      Successfully uninstalled cmake-3.25.2
  Attempting uninstall: decorator
    Found existing installation: decorator 4.4.2
    Uninstalling decorator-4.4.2:
      Successfully uninstalled decorator-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [ ]:
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import networkx as nx
import random
import csv

In [ ]:
##2023-files
events_file_path = "https://drive.google.com/uc?id=1NFN0XGFLixnkIgL5TU7joEIi9WbOsGZE&export=download" #sample file
model_train_file_path = "https://drive.google.com/uc?id=1h2ti-z-trZOHfZiUJTCSbTW5CeowZVxz&export=download"
colab_users_url = "https://drive.google.com/uc?id=1k6FtBdAaCGTiTJKrzDFURfkNMqiCx4cr&export=download" #2023 file with home location
colab_followers_url = "https://drive.google.com/uc?id=1GVIcUexwK0T83QIamzbbUtALFTviB-t8&export=download"

# Load Data

In [ ]:
colab_followers = pd.read_csv(colab_followers_url)
print(colab_followers.shape)
print(colab_followers.head(1))

(160052, 5)
   source  target        created_at        updated_at deleted_at
0       1      70  2014-03-28 18:21  2014-03-28 18:21        NaN


In [ ]:
colab_users = pd.read_csv(colab_users_url)
print(colab_users.head(1))

   id gender  birth_date  city_id city_name  state_id  state_name  created_at  \
0   1   male  1982-06-28     2916    Recife        16  Pernambuco  03/03/2013   

  last_sign_in_at device     lat        lng  
0      12/05/2020     -1 -7.8945 -34.883833  


In [ ]:
colab_users

,id,gender,birth_date,city_id,city_name,state_id,state_name,created_at,last_sign_in_at,device,lat,lng
0,1,male,1982-06-28,2916,Recife,16,Pernambuco,03/03/2013,12/05/2020,-1,-7.894500,-34.883833
1,2,male,1984-06-01,2916,Recife,16,Pernambuco,03/03/2013,17/05/2022,-1,-15.414183,-40.478728
2,3,male,1985-04-29,2916,Recife,16,Pernambuco,03/03/2013,07/11/2021,-1,-8.041835,-34.906227
3,11,male,1976-11-16,2916,Recife,16,Pernambuco,03/03/2013,21/03/2019,-1,-8.029621,-34.921550
4,12,male,1984-03-15,2916,Recife,16,Pernambuco,03/03/2013,21/03/2019,-1,-8.155748,-34.912054
...,...,...,...,...,...,...,...,...,...,...,...,...
50760,690533,female,1999-05-30,3663,São Gonçalo,19,Rio de Janeiro,19/05/2022,19/05/2022,Dart/2.15 (dart:io),-22.825651,-43.035846
50761,690540,female,1979-08-25,5317,Santo André,26,São Paulo,19/05/2022,19/05/2022,Dart/2.15 (dart:io),-23.647685,-46.544777
50762,690553,male,1995-05-14,3663,São Gonçalo,19,Rio de Janeiro,19/05/2022,19/05/2022,Dart/2.15 (dart:io),-22.825651,-43.035846
50763,690555,female,1976-11-05,5317,Santo André,26,São Paulo,19/05/2022,19/05/2022,Dart/2.15 (dart:io),-23.644866,-46.537499


In [ ]:
# Extract unique user IDs from source and target columns
unique_user_ids = set(colab_followers['source']).union(set(colab_followers['target']))
print("Total Unique users", len(unique_user_ids))

# Check for non-existing IDs
non_existing_ids = unique_user_ids - set(colab_users['id'])
print("Total Missing Ids", len(non_existing_ids))

# Convert non-existing IDs to a comma-separated string
non_existing_ids_string = ', '.join(str(id) for id in non_existing_ids)

# Print non-existing IDs as a single string
print("Non-existing user IDs: " + non_existing_ids_string)

# Create DataFrame of non-existing IDs
non_existing_df = pd.DataFrame({'non_existing_ids': list(non_existing_ids)})

# Save DataFrame as CSV
non_existing_df.to_csv('non_existing_ids.csv', index=False)

Total Unique users 62548
Total Missing Ids 38169
Non-existing user IDs: 131073, 4, 5, 6, 7, 8, 9, 10, 131080, 13, 14, 15, 16, 17, 19, 20, 131092, 22, 23, 25, 26, 27, 131098, 29, 30, 131102, 131104, 34, 35, 36, 37, 40, 41, 131113, 43, 44, 46, 48, 49, 50, 51, 52, 131124, 54, 55, 131125, 59, 60, 62, 64, 65, 66, 67, 69, 131142, 74, 75, 76, 77, 79, 82, 262226, 84, 85, 131154, 87, 88, 131155, 90, 131162, 92, 93, 94, 95, 131165, 97, 98, 99, 100, 101, 131167, 103, 104, 131172, 131179, 109, 110, 131183, 112, 113, 131184, 115, 116, 118, 131191, 120, 122, 123, 124, 125, 126, 128, 129, 131, 132, 131205, 131206, 135, 136, 139, 143, 144, 145, 146, 147, 131220, 149, 150, 152, 153, 155, 131227, 157, 158, 262302, 160, 131228, 131230, 131231, 131233, 165, 131235, 131237, 168, 170, 131242, 173, 174, 176, 179, 180, 182, 183, 186, 187, 188, 191, 192, 193, 196, 131269, 199, 200, 202, 204, 205, 131277, 207, 208, 209, 131282, 213, 214, 217, 218, 262363, 221, 222, 223, 225, 227, 229, 230, 231, 232, 235, 236, 1

# (NEW) Subsample from edgelist

In [ ]:
SUBSAMPLE_BY = 0.1
def sample_subgraph(G, fraction):
    # Identify strongly connected components
    strongly_connected_components = nx.strongly_connected_components(G)

    # Find the largest strongly connected component
    largest_component = max(strongly_connected_components, key=len)

    # Calculate the initial number of nodes to sample
    initial_sample_size = int(len(largest_component) * fraction)

    # Randomly sample nodes from the largest component
    initial_sample = random.sample(largest_component, initial_sample_size)

    # Add all neighbors of the sampled nodes to the subgraph
    subgraph_nodes = set(initial_sample)
    for node in initial_sample:
        subgraph_nodes.update(G.neighbors(node))

    # Create a subgraph with the selected nodes
    subgraph = G.subgraph(subgraph_nodes)

    return subgraph
### CREATING GRAPH
G = nx.from_pandas_edgelist(colab_followers, source='source', target='target', create_using=nx.DiGraph())
print("Number of edges in G:", G.number_of_edges())
print("Number of nodes in G:", G.number_of_nodes())
### SUBSAMPLING
sub = sample_subgraph(G, SUBSAMPLE_BY)
sub_fraction = sub.number_of_nodes() / G.number_of_nodes()
print("Number of edges in subsampled G:", sub.number_of_edges())
print("Number of nodes in subsampled G:", sub.number_of_nodes())
####
print("Fraction:", sub_fraction)
# Filtering the colab_users dataset
subsampled_nodes = list(sub.nodes)
subsampled_colab_users = colab_users[colab_users['id'].isin(subsampled_nodes)].copy()

# Create edges_df with subsampled data
edges_df = pd.DataFrame(sub.edges(), columns=['source', 'target'])

# Create nodes_df with subsampled data
source_nodes = edges_df['source'].unique()
target_nodes = edges_df['target'].unique()
node_ids = set().union(source_nodes, target_nodes)
nodes_df = colab_users[colab_users['id'].isin(node_ids)].copy()
print("Length of nodes_df:", len(nodes_df))

# Checking for missing nodes in the subsampled edges
missing_nodes = node_ids - set(nodes_df['id'])

if not missing_nodes:
    print("Validation successful. All source and target nodes have corresponding rows in the nodes dataset.")
else:
    print("Validation failed. The following nodes are missing in the nodes dataset:", len(missing_nodes))
    print(sorted(missing_nodes))
###
edges_df.to_csv('ergm_sampled_10_pct_edges.csv', index=False)
nodes_df.to_csv('ergm_sampled_10_pct_nodes.csv', index=False)

Number of edges in G: 160052
Number of nodes in G: 62548


<ipython-input-45-e465b8c49ebd>:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  initial_sample = random.sample(largest_component, initial_sample_size)


Number of edges in subsampled G: 36370
Number of nodes in subsampled G: 8751
Fraction: 0.13990855023342072
Length of nodes_df: 4030
Validation failed. The following nodes are missing in the nodes dataset: 4721
[6, 10, 14, 15, 17, 19, 22, 26, 34, 36, 40, 41, 43, 44, 46, 48, 49, 51, 52, 62, 65, 66, 69, 74, 76, 84, 85, 95, 99, 104, 109, 112, 113, 120, 122, 128, 136, 143, 145, 150, 153, 155, 160, 165, 170, 187, 188, 192, 199, 202, 209, 223, 229, 231, 232, 240, 244, 246, 259, 263, 264, 271, 273, 274, 277, 279, 282, 288, 289, 290, 293, 297, 303, 304, 320, 323, 329, 345, 358, 359, 375, 378, 380, 382, 384, 392, 393, 395, 398, 400, 401, 416, 417, 433, 438, 447, 456, 461, 465, 471, 480, 483, 484, 487, 502, 504, 506, 510, 515, 521, 524, 526, 544, 554, 558, 559, 606, 612, 626, 635, 653, 671, 673, 675, 679, 683, 685, 688, 691, 696, 698, 721, 730, 734, 736, 737, 742, 753, 759, 772, 791, 792, 797, 809, 810, 813, 818, 819, 829, 830, 831, 841, 843, 845, 848, 850, 851, 859, 866, 908, 909, 915, 919, 935,

In [ ]:
# Assuming you want to retrieve rows with id 1 and 2
ids_to_retrieve = [16388, 5]

# Retrieve rows from colab_users based on id
selected_rows = colab_users.loc[colab_users['id'].isin(ids_to_retrieve)]

# Print the selected rows
print(selected_rows)

Empty DataFrame
Columns: [id, gender, birth_date, city_id, city_name, state_id, state_name, created_at, last_sign_in_at, device, lat, lng]
Index: []


# Create R input file
This script wil iterate the colab_followers (edgelist) and match with data from nodes creating a single edgelist file

In [ ]:
# Iterate over colab_followers and extract relevant information
result_data = []
for index, row in tqdm(colab_followers.iterrows(), total=len(colab_followers)):
    source_id = row['source']
    # Get location, gender, and birth date from colab_users
    user_info = colab_users.loc[colab_users['id'] == source_id]
    location = user_info['city_name'].values[0]
    gender = user_info['gender'].values[0]
    birth_date = datetime.strptime(user_info['birth_date'].values[0], '%Y-%m-%d')
    # Calculate age based on birth date
    today = datetime.today()
    age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    # Append the result to the data list
    result_data.append([source_id, row['target'], location, gender, age])
#
# Create a new DataFrame from the result_data list
result_df = pd.DataFrame(result_data, columns=['source', 'target', 'location', 'gender', 'age'])
#
# Save the new DataFrame to a CSV file
result_df.to_csv('ergm_input.csv', index=False)

  5%|▍         | 7922/160052 [00:10<03:14, 781.65it/s]


KeyboardInterrupt: ignored

# Subsampling the data

In [ ]:
# Load the network data from the CSV file
df = pd.read_csv("ergm_input.csv")
print(f"Data Frame rows: {len(df)}")
# Create a directed graph from the DataFrame
G = nx.from_pandas_edgelist(df, source='source', target='target', create_using=nx.DiGraph())
print("Number of edges in G:", G.number_of_edges())
print("Number of nodes in G:", G.number_of_nodes())

Data Frame rows: 160052
Number of edges in G: 160052
Number of nodes in G: 62548


In [ ]:
# Remove self-loops
self_loops = list(nx.selfloop_edges(G))
G.remove_edges_from(self_loops)

# Remove isolated nodes
isolated_nodes = list(nx.isolates(G))
G.remove_nodes_from(isolated_nodes)
print("Number of edges in G:", G.number_of_edges())
print("Number of nodes in G:", G.number_of_nodes())
def sample_subgraph(G, fraction):
    # Identify strongly connected components
    strongly_connected_components = nx.strongly_connected_components(G)

    # Find the largest strongly connected component
    largest_component = max(strongly_connected_components, key=len)

    # Calculate the initial number of nodes to sample
    initial_sample_size = int(len(largest_component) * fraction)

    # Randomly sample nodes from the largest component
    initial_sample = random.sample(largest_component, initial_sample_size)

    # Add all neighbors of the sampled nodes to the subgraph
    subgraph_nodes = set(initial_sample)
    for node in initial_sample:
        subgraph_nodes.update(G.neighbors(node))

    # Create a subgraph with the selected nodes
    subgraph = G.subgraph(subgraph_nodes)

    return subgraph
##
sub = sample_subgraph(G, 0.1)
print("Number of edges in sub:", sub.number_of_edges())
print("Number of nodes in sub:", sub.number_of_nodes())

Number of edges in G: 159994
Number of nodes in G: 62538
Number of edges in sub: 39712
Number of nodes in sub: 9178


<ipython-input-44-e69e8fd6318e>:21: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  initial_sample = random.sample(largest_component, initial_sample_size)


In [ ]:
sampled_nodes= sub.nodes()
# Extract the properties for the sampled nodes from the original DataFrame
sampled_properties = df[df['source'].isin(sampled_nodes) & df['target'].isin(sampled_nodes)]

# Create a new DataFrame for the sampled graph with properties
sampled_graph_with_properties = pd.DataFrame(list(sub.edges()), columns=['source', 'target'])
sampled_graph_with_properties = sampled_graph_with_properties.merge(sampled_properties, on=['source', 'target'], how='left')

# Check if the network has loops
has_loops = any(sampled_graph_with_properties["source"] == sampled_graph_with_properties["target"])

# Print the result
if has_loops:
    print("The network contains loops.")
else:
    print("The network does not contain loops.")

# Remove loops if they exist
if has_loops:
    sampled_graph_with_properties = sampled_graph_with_properties[sampled_graph_with_properties["source"] != sampled_graph_with_properties["target"]]
    sampled_graph_with_properties.reset_index(drop=True, inplace=True)
    print("Loops have been removed.")

# Export the sampled graph with properties to a new CSV file
sampled_graph_with_properties.to_csv('ergm_sampled.csv', index=False)
sampled_graph_with_properties

The network does not contain loops.


,source,target,location,gender,age
0,1,70,Recife,male,41
1,1,130,Recife,male,41
2,1,137,Recife,male,41
3,1,289,Recife,male,41
4,1,564,Recife,male,41
...,...,...,...,...,...
39707,131059,88799,Porto Alegre,female,59
39708,131059,90672,Porto Alegre,female,59
39709,65535,46876,Santos,male,45
39710,65535,110893,Santos,male,45


# Unmerging edgelist into nodes and edges files

In [ ]:
df = pd.read_csv("ergm_sampled_10_pct.csv")
df = df[['source', 'target']].drop_duplicates()
G = nx.from_pandas_edgelist(df, source='source', target='target', create_using=nx.DiGraph())
print("Number of edges in G:", G.number_of_edges())
print("Number of nodes in G:", G.number_of_nodes())
# Remove self-loops
self_loops = list(nx.selfloop_edges(G))
G.remove_edges_from(self_loops)

# Remove isolated nodes
isolated_nodes = list(nx.isolates(G))
G.remove_nodes_from(isolated_nodes)
print("Number of edges in G:", G.number_of_edges())
print("Number of nodes in G:", G.number_of_nodes())

Number of edges in G: 39712
Number of nodes in G: 9178
Number of edges in G: 39712
Number of nodes in G: 9178


In [ ]:
df = pd.read_csv("ergm_sampled_10_pct.csv")
print(df.head())
print(f"Original Datasource Length: {len(df)}")
# Create edges.csv with source and target columns
edges_df = df[['source', 'target']].drop_duplicates()
print(edges_df.head())
print(f"Edges Length: {len(edges_df)}")
# Create nodes.csv with unique user_id, location, gender, and age columns
nodes_df = df.drop_duplicates(subset='source')[['source', 'location', 'gender', 'age']]
nodes_df.rename(columns={'source': 'user_id'}, inplace=True)
print(nodes_df.head())
print(f"Nodes Length: {len(nodes_df)}")
# Get unique user_ids from edges_df
unique_user_ids = set(edges_df['source']).union(set(edges_df['target']))

# Find user_ids without corresponding rows in nodes_df
invalid_user_ids = [user_id for user_id in unique_user_ids if user_id not in nodes_df['user_id'].values]

if not invalid_user_ids:
    print("Validation successful. All user_ids have corresponding rows in nodes_df.")
else:
    print("Validation failed. The following user_ids do not have corresponding rows in nodes_df:")
    print(invalid_user_ids)
#
#edges_df.to_csv('ergm_sampled_10_pct_edges.csv', index=False)
#nodes_df.to_csv('ergm_sampled_10_pct_nodes.csv', index=False)

   source  target location gender  age
0       1      70   Recife   male   41
1       1     130   Recife   male   41
2       1     137   Recife   male   41
3       1     289   Recife   male   41
4       1     564   Recife   male   41
Original Datasource Length: 39712
   source  target
0       1      70
1       1     130
2       1     137
3       1     289
4       1     564
Edges Length: 39712
     user_id      location  gender  age
0          1        Recife    male   41
23         2        Recife    male   39
156    32770  Porto Alegre    male   33
163        3        Recife    male   38
252   131075       Niterói  female   59
Nodes Length: 3016
Validation failed. The following user_ids do not have corresponding rows in nodes_df:
[5, 6, 7, 8, 32772, 10, 13, 32782, 229390, 16, 17, 19, 20, 22, 23, 25, 26, 30, 65568, 34, 36, 37, 40, 65579, 44, 46, 360494, 32816, 48, 49, 196657, 52, 32817, 55, 131127, 163888, 262202, 360511, 65, 66, 65601, 69, 74, 75, 76, 65609, 78, 82, 84, 262226, 87, 19

In [ ]:
colab_users = pd.read_csv(colab_users_url)
print(colab_users.head(1))
# Find user_ids without corresponding rows in nodes_df
invalid_user_ids = [user_id for user_id in unique_user_ids if user_id not in colab_users['id'].values]

if not invalid_user_ids:
    print("Validation successful. All user_ids have corresponding rows in nodes_df.")
else:
    print("Validation failed. The following user_ids do not have corresponding rows in nodes_df:")
    print(len(invalid_user_ids))

   id gender  birth_date  city_id city_name  state_id  state_name  created_at  \
0   1   male  1982-06-28     2916    Recife        16  Pernambuco  03/03/2013   

  last_sign_in_at device     lat        lng  
0      12/05/2020     -1 -7.8945 -34.883833  
Validation failed. The following user_ids do not have corresponding rows in nodes_df:
4670
